In [75]:
import os
import sys
import pandas as pd

In [76]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [231]:
from functions import get_dates, get_accesstoken

In [78]:
from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [266]:
accesstoken = get_accesstoken(client_id_1, client_secret_1, refresh_token_1, client_version, token_url)

In [267]:
token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [278]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import calendar

def get_months_dates(month, year):
    # This returns list of dates of specified lengths of months.
    lst = []
    for i in range(-9, calendar.monthlen(year, month) + 1): 
        date_ = datetime.date(year, month, 1)
        date = date_ + datetime.timedelta(days=i)
        lst.append(date.strftime("%m/%d/%Y 00:00:00"))
    return lst

In [279]:
month = 1
year = 2019
days = get_months_dates(month, year)

In [280]:
print(days)

['12/23/2018 00:00:00', '12/24/2018 00:00:00', '12/25/2018 00:00:00', '12/26/2018 00:00:00', '12/27/2018 00:00:00', '12/28/2018 00:00:00', '12/29/2018 00:00:00', '12/30/2018 00:00:00', '12/31/2018 00:00:00', '01/01/2019 00:00:00', '01/02/2019 00:00:00', '01/03/2019 00:00:00', '01/04/2019 00:00:00', '01/05/2019 00:00:00', '01/06/2019 00:00:00', '01/07/2019 00:00:00', '01/08/2019 00:00:00', '01/09/2019 00:00:00', '01/10/2019 00:00:00', '01/11/2019 00:00:00', '01/12/2019 00:00:00', '01/13/2019 00:00:00', '01/14/2019 00:00:00', '01/15/2019 00:00:00', '01/16/2019 00:00:00', '01/17/2019 00:00:00', '01/18/2019 00:00:00', '01/19/2019 00:00:00', '01/20/2019 00:00:00', '01/21/2019 00:00:00', '01/22/2019 00:00:00', '01/23/2019 00:00:00', '01/24/2019 00:00:00', '01/25/2019 00:00:00', '01/26/2019 00:00:00', '01/27/2019 00:00:00', '01/28/2019 00:00:00', '01/29/2019 00:00:00', '01/30/2019 00:00:00', '01/31/2019 00:00:00', '02/01/2019 00:00:00']


In [281]:
import datetime
import requests

In [282]:
def json_get_sales_count(obj):
    """Recursively fetch values from nested JSON."""
    arr = {'bakedgoods': {}, 'breakfast': {}, 'lunch': {}}
    itemvalue = None
    categoryname = None
    itemname = None

    def extract(obj, arr, itemvalue, categoryname, itemname):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, itemvalue, categoryname, itemname)
                elif k == 'item':
                    itemkey = v
                elif k == 'soldQuantity':
                    itemvalue = v
                elif k == 'categoryName':
                    if v == 'LUNCH':
                        categoryname = 'lunch'
                    elif v == 'BREAKFAST':
                        categoryname = 'breakfast'
                    elif v == 'BAKED GOODS':
                        categoryname = 'bakedgoods'
                elif k == 'productName':
                    v = v.replace(' + ', '')
                    v = v.replace('To Go', '')
                    v = v.replace('Sit In', '')
                    v = v.replace('/w Mixed Seeds, Nuts & Honey', '')
                    v = v.replace('/w Honey', '')
                    v = v.replace(', ', '')
                    v = v.replace('Not Heated', '')
                    v = v.replace('Heated', '')
                    v = v.replace('Soup & Sandwich', '')
                    v = v.replace('Ham Toastie', 'Ham & Cheese Toastie')
                    itemname = v

                if categoryname and itemvalue and itemname:
                    if itemname in arr[categoryname]:
                        arr[categoryname][itemname] += itemvalue
                    else:
                        arr[categoryname][itemname] = itemvalue
                    itemvalue = None
                    categoryname = None
                    itemname = None

        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, itemvalue, categoryname, itemname)
        return arr

    values = extract(obj, arr, itemvalue, categoryname, itemname)
    return values

In [283]:
def get_summary_report(token, startdate, enddate):
    arr = {
        'searchCriteria': {
            'startDate': startdate,
            'endDate': enddate,
            'includedReports': [102]
        }
    }
    result = requests.post(url='https://mapi-eu.talech.com/reports/receiptssummaryreport',
                           json=arr,
                           headers=token)
    items = result.json()
    salesdata = json_get_sales_count(items)
    return salesdata
    return items

In [284]:
dailysales = {}
for i in range(0, len(days)):
    d = pd.to_datetime(days[i])
    end = str(d.replace(hour=23).strftime('%m/%d/%Y %H:%M:%S'))
    salesdata = get_summary_report(token, days[i], end)
    dailysales[days[i]] = salesdata

In [285]:
dailysales

{'12/23/2018 00:00:00': {'bakedgoods': {'Festive Financier': 10,
   'Custard Tart': 10,
   'Mixed Berry Scone': 17,
   'Pain au Chocolat': 6,
   'Anzac Biscuit': 9,
   'Pain au Raisin': 7,
   'Wholemeal Muffin': 5,
   'Oatmeal & Raisin Cookie': 3,
   'Cinnabun': 15,
   'Coconut Snowball': 8,
   'Croissant': 7,
   'Peanut Butter Brownie': 4,
   'Snickers Boost Ball': 4},
  'breakfast': {'Sausage Roll': 10,
   'Ham Cheese Croissant': 1,
   'Berry & Granola Parfait': 1},
  'lunch': {'Turkey Sandwich': 2,
   'Ham & Cheese Toastie': 2,
   'Mediterranean Vegan Wrap': 1,
   'Chicken Bacon Wrap': 3,
   'Chilli Tuna Wrap': 1}},
 '12/24/2018 00:00:00': {'bakedgoods': {'Festive Financier': 11,
   'Custard Tart': 10,
   'Peanut Butter Brownie': 2,
   'Pain au Chocolat': 10,
   'Anzac Biscuit': 19,
   'Mixed Berry Scone': 28,
   'Pain au Raisin': 10,
   'Wholemeal Muffin': 11,
   'Oatmeal & Raisin Cookie': 7,
   'Cinnabun': 10,
   'Coconut Snowball': 1,
   'Croissant': 12,
   'Snickers Boost Ball':

In [286]:
dailysales_df = pd.DataFrame(dailysales)

In [287]:
dailysales_df

,12/23/2018 00:00:00,12/24/2018 00:00:00,12/25/2018 00:00:00,12/26/2018 00:00:00,12/27/2018 00:00:00,12/28/2018 00:00:00,12/29/2018 00:00:00,12/30/2018 00:00:00,12/31/2018 00:00:00,01/01/2019 00:00:00,...,01/23/2019 00:00:00,01/24/2019 00:00:00,01/25/2019 00:00:00,01/26/2019 00:00:00,01/27/2019 00:00:00,01/28/2019 00:00:00,01/29/2019 00:00:00,01/30/2019 00:00:00,01/31/2019 00:00:00,02/01/2019 00:00:00
bakedgoods,"{'Festive Financier': 10, 'Custard Tart': 10, ...","{'Festive Financier': 11, 'Custard Tart': 10, ...",{},{},"{'Festive Financier': 7, 'Coconut Snowball': 3...","{'Festive Financier': 8, 'Custard Tart': 8, 'M...","{'Festive Financier': 8, 'Coconut Snowball': 1...","{'Festive Financier': 4, 'Coconut Snowball': 1...","{'Festive Financier': 3, 'Coconut Snowball': 3...",{},...,"{'Mixed Berry Scone': 22, 'Cinnabun': 7, 'Pain...","{'Mixed Berry Scone': 25, 'Cinnabun': 8, 'Oatm...","{'Mixed Berry Scone': 31, 'Cinnabun': 9, 'Pain...","{'Mixed Berry Scone': 9, 'Cinnabun': 7, 'Pain ...","{'Mixed Berry Scone': 12, 'Cinnabun': 5, 'Pain...","{'Mixed Berry Scone': 15, 'Cinnabun': 6, 'Pain...","{'Mixed Berry Scone': 23, 'Cinnabun': 4, 'Pain...","{'Mixed Berry Scone': 20, 'Cinnabun': 7, 'Pain...","{'Mixed Berry Scone': 19, 'Cinnabun': 7, 'Pain...","{'Mixed Berry Scone': 27, 'Cinnabun': 9, 'Pain..."
breakfast,"{'Sausage Roll': 10, 'Ham Cheese Croissant': 1...","{'Sausage Roll': 8, 'Ham Cheese Croissant': 7,...",{},{},"{'Sausage Roll': 6, 'Granary Toast': 1, 'Berry...","{'Sausage Roll': 10, 'Berry & Granola Parfait'...","{'Sausage Roll': 4, 'Granary Toast': 1}",{'Sausage Roll': 5},"{'Sausage Roll': 3, 'Berry & Granola Parfait':...",{},...,"{'Sourdough Toast': 3, 'Sausage Roll': 4, 'Org...","{'Sourdough Toast': 6, 'Sausage Roll': 9, 'Org...","{'Sourdough Toast': 3, 'Sausage Roll': 7, 'Org...","{'Sausage Roll': 2, 'Ham Cheese Croissant': 2,...","{'Sausage Roll': 4, 'Ham Cheese Croissant': 3,...","{'Sourdough Toast': 2, 'Sausage Roll': 3, 'Org...","{'Sourdough Toast': 2, 'Sausage Roll': 2, 'Org...","{'Sourdough Toast': 4, 'Sausage Roll': 3, 'Org...","{'Sourdough Toast': 5, 'Sausage Roll': 5, 'Org...","{'Sourdough Toast': 2, 'Sausage Roll': 7, 'Org..."
lunch,"{'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...","{'Caprese': 2, 'Turkey Sandwich': 3, 'Ham & Ch...",{},{},"{'Mediterranean Vegan Wrap': 1, 'Ham & Cheese ...","{'Caprese': 4, 'Turkey Sandwich': 3, 'Ham & Ch...","{'Chicken Bacon Wrap': 4, 'Turkey Sandwich': 2...","{'Caprese': 1, 'Mediterranean Vegan Wrap': 1, ...","{'Ham & Cheese Toastie': 5, 'Chicken Bacon Wra...",{},...,"{'Mediterranean Vegan Wrap': 1, 'Chicken Bacon...","{'Caprese': 1, 'Ham & Cheese Toastie': 2, 'Tur...","{'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...","{'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...",{},"{'Turkey Sandwich': 2, 'Chicken Bacon Wrap': 2...","{'Caprese': 3, 'Mediterranean Vegan Wrap': 2, ...","{'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...","{'Caprese': 2, 'Turkey Sandwich': 3, 'Ham & Ch...","{'Ham & Cheese Toastie': 2, 'Chicken Bacon Wra..."


In [288]:
dailysales_df.loc['bakedgoods', '01/02/2019 00:00:00']

{'Mixed Berry Scone': 17,
 'Pain au Chocolat': 5,
 'Croissant': 5,
 'Oatmeal & Raisin Cookie': 3,
 'Custard Tart': 4,
 'Cinnabun': 5,
 'Anzac Biscuit': 9,
 'Snickers Boost Ball': 8,
 'Almond Financier': 1,
 'Coconut Snowball': 3,
 'Pain au Raisin': 8,
 'Peanut Butter Brownie': 2,
 'Wholemeal Muffin': 3,
 'Festive Financier': 1}

In [289]:
from dataclasses import dataclass

In [290]:
class SalesGrowth:
    def __init__(self, category, day, percentile):
        self.category = category
        self.day = day
        self.percentile = percentile
        
    def __iter__(self):
        yield self.category
        yield self.day
        yield self.percentile
        
    @classmethod
    def columns(cls):
        return['category', 'day', 'percentile']

In [291]:
class AllSalesGrowth:
    def __init__(self):
        self.lst = []
        
    def set_item(self, item):
        self.lst.append(item)
        
    def get_item(self, category, day):
        for i in range(len(self.lst)):
            if self.lst[i].category == category and self.lst[i].day == day:
                return self.lst[i]
        return False
    
    def get_percentile(self, category, day):
        for i in range(len(self.lst)):
            if self.lst[i].category == category and self.lst[i].day == day:
                    return self.lst[i].percentile
                
    @property
    def items(self):
        return pd.DataFrame(self.lst, columns=SalesGrowth.columns())
                

In [292]:
import datetime

In [293]:
growth_json = pd.read_json('data/percentile_change/{}_{}.json'.format(month, year))
growth_json

,0,1,2,3,4,5,6
bakedgoods,14.3,12.4,8.6,6.0,1.5,-2.0,24.0
breakfast,50.0,41.3,48.8,45.8,47.2,34.3,43.9
lunch,42.1,46.4,46.4,43.0,46.0,23.4,-16.1


In [294]:
growths = AllSalesGrowth()

for category in growth_json.index:
    for day in growth_json.columns:    
        percentile = growth_json.loc[category, day]
        growth = SalesGrowth(category, day, percentile)
        growths.set_item(growth)

In [295]:
growths.items

,category,day,percentile
0,bakedgoods,0,14.3
1,bakedgoods,1,12.4
2,bakedgoods,2,8.6
3,bakedgoods,3,6.0
4,bakedgoods,4,1.5
5,bakedgoods,5,-2.0
6,bakedgoods,6,24.0
7,breakfast,0,50.0
8,breakfast,1,41.3
9,breakfast,2,48.8


In [296]:
weathers_json = pd.read_json('data/historical_weather/{}_{}.json'.format(month, year))
weathers_json

,temperature,wind_speed,weather_code,precip,MM/DD/YY,weeknum,dayofweek,hour,date,month,year
2019-01-01 06:00:00,7,17,116,0.0,01/01/2019,0,1,6,1,1,2019
2019-01-01 12:00:00,9,11,119,0.0,01/01/2019,0,1,12,1,1,2019
2019-01-02 06:00:00,6,8,116,0.0,01/02/2019,0,2,6,2,1,2019
2019-01-02 12:00:00,8,14,119,0.0,01/02/2019,0,2,12,2,1,2019
2019-01-03 06:00:00,4,7,116,0.0,01/03/2019,0,3,6,3,1,2019
2019-01-03 12:00:00,7,13,176,0.1,01/03/2019,0,3,12,3,1,2019
2019-01-04 06:00:00,4,10,113,0.0,01/04/2019,0,4,6,4,1,2019
2019-01-04 12:00:00,6,8,113,0.0,01/04/2019,0,4,12,4,1,2019
2019-01-05 06:00:00,5,14,116,0.0,01/05/2019,0,5,6,5,1,2019
2019-01-05 12:00:00,8,15,116,0.0,01/05/2019,0,5,12,5,1,2019


In [325]:
class Weather:
    def __init__(self, date, temp, wind, precip):
        self.date = date
        self.weather_m = 'null'
        self.weather_a = 'null'
        self.temp = temp
        self.wind = wind
        self.precip = precip
        
    def __iter__(self):
        yield self.date
        yield self.weather_m
        yield self.weather_a
        yield self.temp
        yield self.wind
        yield self.precip
        
    @classmethod
    def columns(cls):
        return['date', 'weather_m', 'weather_a', 'temp', 'wind', 'precip']
        

In [326]:
class AllWeather:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.date] = item
        
    def get_item(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
    
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Weather.columns())

In [327]:
weathers = AllWeather()

for datetime in weathers_json.index:
    date = weathers_json.loc[datetime, 'MM/DD/YY']
    temp = weathers_json.loc[datetime, 'temperature']
    wind = weathers_json.loc[datetime, 'wind_speed']
    precip = weathers_json.loc[datetime, 'precip']
    hour = weathers_json.loc[datetime, 'hour']
    weather_code = weathers_json.loc[datetime, 'weather_code']
    weather = weathers.get_item(date)
    if weather == False:
        weather = Weather(date, temp, wind, precip)
        weather.weather_m = weather_code
        weathers.set_item(weather)
    else:
        weather.weather_a = weather_code

In [328]:
weathers.items

,date,weather_m,weather_a,temp,wind,precip
0,01/01/2019,116,119,7,17,0.0
1,01/02/2019,116,119,6,8,0.0
2,01/03/2019,116,176,4,7,0.0
3,01/04/2019,113,113,4,10,0.0
4,01/05/2019,116,116,5,14,0.0
5,01/06/2019,122,176,7,10,0.0
6,01/07/2019,122,119,10,21,0.0
7,01/08/2019,116,116,6,23,0.0
8,01/10/2019,143,266,8,9,0.1
9,01/11/2019,176,116,7,18,0.1


In [329]:
class Date:
    def __init__(self, date):
        self.date = date
        self.dayofweek = pd.Timestamp(pd.to_datetime(date)).dayofweek
        self.weeknum = pd.to_datetime(date).strftime("%W")
        self.day = pd.to_datetime(date).day
        self.month = pd.to_datetime(date).month
        self.year = pd.to_datetime(date).year
        self.holiday = 0
        
    def __iter__(self):
        yield self.dayofweek
        yield self.weeknum
        yield self.day
        yield self.month
        yield self.year
        yield self.holiday
        
    @classmethod
    def columns(cls):
        return['dayofweek', 'weeknum', 'day', 'month', 'year', 'holiday']
        
class Calendar:
    def __init__(self):
        self.arr = {}
        
    def set_date(self, item):
        self.arr[item.date] = item
        
    def get_date(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Date.columns())

In [330]:
class Product:
    __lastid = 1
    
    def __init__(self, name, category):
        self.id = Product.__lastid
        self.name = name
        self.category = category
        
        Product.__lastid += 1
        
    def __iter__(self):
        yield self.id
        yield self.name
        yield self.category
    
    @classmethod
    def columns(cls):
        return ['id', 'name', 'category']
        
    
class Sales:
    def __init__(self, soldnum, date=None, product=None, weather=None, growth=None):
        self.product = product
        self.date = date
        self.soldnum = soldnum
        self.prevdaysales = []
        self.weather = weather
        self.growth = growth
        
    def set_prevdaysales(self, quantity):
        self.prevdaysales.append(quantity)
        
    def __iter__(self):
        yield self.product.id
        yield self.product.name
        yield self.product.category
        yield self.date.date
        yield self.date.dayofweek
        yield self.date.weeknum
        yield self.date.year
        yield self.date.holiday
        yield self.soldnum
        for i in range(7):
            try:
                prevsales = self.prevdaysales[i]
            except:
                prevsales = 'null'
            yield prevsales
            
        yield self.weather.weather_m
        yield self.weather.weather_a
        yield self.weather.temp
        yield self.weather.wind
        yield self.weather.precip
        
    @classmethod
    def columns(cls):
        return['id', 'name', 'category', 'date', 'dayofweek', 'weeknum', 'year',
               'holiday', 'soldnum', 'prevdaysales1', 'prevdaysales2', 'prevdaysales3', 
               'prevdaysales4', 'prevdaysales5', 'prevdaysales6', 'prevdaysales7',
               'weather_morning', 'weather_afternoon', 'temp', 'wind', 'precip']

In [331]:
class AllProduct:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.name] = item
        
    def get_item(self, name):
        if name in self.arr:
            return self.arr[name]
        else:
            return False
        
    def delete(self, name):
        del self.arr[name]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Product.columns())

In [332]:
class Trainingset:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        key = '{}, {}'.format(item.date.date, item.product.name)
        self.arr[key] = item
        
    def get_item(self, name, date):
        key = '{}, {}'.format(date, name)
        if key in self.arr:
            return self.arr[key]
        else:
            return False
    
    def delete(self, name, date):
        key = '{}, {}'.format(date, name)
        del self.arr[key]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Sales.columns())

In [333]:
url = 'https://holidays.abstractapi.com/v1/'
params = {
    'api_key': '02a0112703a34f5abb1ab898b4f8125a',
    'country': 'IE',
    'year': '2019'
}

In [334]:
result = requests.get(url=url, params=params)
result_json = result.json()
df_holidays = pd.DataFrame(result_json)
df_holidays

,name,name_local,language,description,country,location,state,is_observed,type,date,date_year,date_month,date_day,week_day
0,New Year's Day,,,,IE,,,False,public_holiday,01/01/2019,2019,01,01,Tuesday
1,St. Patrick's Day,,,,IE,,,False,public_holiday,03/17/2019,2019,03,17,Sunday
2,Easter Monday,,,,IE,,,False,public_holiday,04/22/2019,2019,04,22,Monday
3,May Day,,,,IE,,,False,public_holiday,05/06/2019,2019,05,06,Monday
4,June Bank Holiday,,,,IE,,,False,public_holiday,06/03/2019,2019,06,03,Monday
5,August Bank Holiday,,,,IE,,,False,public_holiday,08/05/2019,2019,08,05,Monday
6,October Bank Holiday,,,,IE,,,False,public_holiday,10/28/2019,2019,10,28,Monday
7,Christmas Day,,,,IE,,,False,public_holiday,12/25/2019,2019,12,25,Wednesday
8,St. Stephen's Day,,,,IE,,,False,public_holiday,12/26/2019,2019,12,26,Thursday


In [347]:
trset = Trainingset()
products = AllProduct()
calendar = Calendar()
dt_ = None
holiday = 0

# if all categories are True, that means the shop
# sold none of products, which means the shop were closed
closed = {'bakedgoods': False, 'lunch': False, 'breakfast': False} 

for date in dailysales_df.columns:
    dt_ = pd.to_datetime(date).strftime("%m/%d/%Y")
    dt = Date(dt_)

    for index in df_holidays.index:
        if df_holidays.loc[index, 'date'] == dt_:
            dt.holiday = 1
    
    calendar.set_date(dt)
    
    for category in dailysales_df.index:
        sales = dailysales_df.loc[category, date]
        if sales == {}:
            closed[category] = True
            
        for itemname, sales in sales.items():
            product = products.get_item(itemname)
            weather = weathers.get_item(dt_)
            
            if product == False:
                product = Product(itemname, category)
                products.set_item(product)
            
            if weather == False:
                weather = Weather(dt_, 'null', 'null', 'null')
                weathers.set_item(weather)
            
            dateinfo = calendar.get_date(dt_)
            
            growth = growths.get_item(product.category, dateinfo.dayofweek)
            sales = Sales(sales, dateinfo, product, weather, growth)
            trset.set_item(sales)

    if closed['bakedgoods'] and closed['lunch'] and closed['breakfast']:
        # if shops seems closed, do nothing
        closed['bakedgoods'] = False
        closed['lunch'] = False
        closed['breakfast'] = False
        continue
    else:
        # if the shop is open but seems the product sales is 0,
        # add the sales 0 into the training data
        for index in products.items.index:
            itemname = products.items.loc[index, 'name']
            item = trset.get_item(itemname, dt_)
            if item is False:
                dateinfo = calendar.get_date(dt_)
                product = products.get_item(itemname)
                weather = weathers.get_item(dt_)
                print(weather)
                
                if weather == False:
                    weather = Weather(dt_, 'null', 'null', 'null')
                    weathers.set_item(weather)
                    
                growth = growths.get_item(product.category, dateinfo.dayofweek)
                sales = Sales(0, dateinfo, product, weather, growth)
                trset.set_item(sales)

In [348]:
weathers.items

,date,weather_m,weather_a,temp,wind,precip
0,01/01/2019,116,119,7,17,0
1,01/02/2019,116,119,6,8,0
2,01/03/2019,116,176,4,7,0
3,01/04/2019,113,113,4,10,0
4,01/05/2019,116,116,5,14,0
5,01/06/2019,122,176,7,10,0
6,01/07/2019,122,119,10,21,0
7,01/08/2019,116,116,6,23,0
8,01/10/2019,143,266,8,9,0.1
9,01/11/2019,176,116,7,18,0.1


In [349]:
trset.items

,id,name,category,date,dayofweek,weeknum,year,holiday,soldnum,prevdaysales1,...,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,prevdaysales7,weather_morning,weather_afternoon,temp,wind,precip
0,65,Festive Financier,bakedgoods,12/23/2018,6,51,2018,0,10,null,...,null,null,null,null,null,null,null,null,null,null
1,66,Custard Tart,bakedgoods,12/23/2018,6,51,2018,0,10,null,...,null,null,null,null,null,null,null,null,null,null
2,67,Mixed Berry Scone,bakedgoods,12/23/2018,6,51,2018,0,17,null,...,null,null,null,null,null,null,null,null,null,null
3,68,Pain au Chocolat,bakedgoods,12/23/2018,6,51,2018,0,6,null,...,null,null,null,null,null,null,null,null,null,null
4,69,Anzac Biscuit,bakedgoods,12/23/2018,6,51,2018,0,9,null,...,null,null,null,null,null,null,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,92,Cheese Croissant,breakfast,02/01/2019,4,04,2019,0,0,null,...,null,null,null,null,null,null,null,null,null,null
1058,93,Cruffin,bakedgoods,02/01/2019,4,04,2019,0,0,null,...,null,null,null,null,null,null,null,null,null,null
1059,94,Chicken Serrano Roll,lunch,02/01/2019,4,04,2019,0,0,null,...,null,null,null,null,null,null,null,null,null,null
1060,95,Chicken Serrano,lunch,02/01/2019,4,04,2019,0,0,null,...,null,null,null,null,null,null,null,null,null,null


In [338]:
products.items

,id,name,category
0,1,Festive Financier,bakedgoods
1,2,Custard Tart,bakedgoods
2,3,Mixed Berry Scone,bakedgoods
3,4,Pain au Chocolat,bakedgoods
4,5,Anzac Biscuit,bakedgoods
5,6,Pain au Raisin,bakedgoods
6,7,Wholemeal Muffin,bakedgoods
7,8,Oatmeal & Raisin Cookie,bakedgoods
8,9,Cinnabun,bakedgoods
9,10,Coconut Snowball,bakedgoods


In [339]:
from datetime import datetime, timedelta

In [358]:
for index in trset.items.index:
    count = 0
    currentitem = trset.get_item(trset.items.loc[index, 'name'], trset.items.loc[index, 'date'])
    currentdt = pd.to_datetime(currentitem.date.date)
    name = currentitem.product.name
    
    while len(currentitem.prevdaysales) < 7:
        if count < -20:
            break
        count -= 1
        
        prevdt = currentdt + timedelta(days=count)
        prevdt = prevdt.strftime("%m/%d/%Y")
        previtem = trset.get_item(name, prevdt)
        if previtem is not False:
            currentitem.set_prevdaysales(previtem.soldnum)
        else:
            continue
    count = 0

In [359]:
trset.items

,id,name,category,date,dayofweek,weeknum,year,holiday,soldnum,prevdaysales1,...,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,prevdaysales7,weather_morning,weather_afternoon,temp,wind,precip
0,65,Festive Financier,bakedgoods,12/23/2018,6,51,2018,0,10,null,...,null,null,null,null,null,null,null,null,null,null
1,66,Custard Tart,bakedgoods,12/23/2018,6,51,2018,0,10,null,...,null,null,null,null,null,null,null,null,null,null
2,67,Mixed Berry Scone,bakedgoods,12/23/2018,6,51,2018,0,17,null,...,null,null,null,null,null,null,null,null,null,null
3,68,Pain au Chocolat,bakedgoods,12/23/2018,6,51,2018,0,6,null,...,null,null,null,null,null,null,null,null,null,null
4,69,Anzac Biscuit,bakedgoods,12/23/2018,6,51,2018,0,9,null,...,null,null,null,null,null,null,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,92,Cheese Croissant,breakfast,02/01/2019,4,04,2019,0,0,0,...,0,0,0,0,0,null,null,null,null,null
1058,93,Cruffin,bakedgoods,02/01/2019,4,04,2019,0,0,0,...,0,0,0,0,0,null,null,null,null,null
1059,94,Chicken Serrano Roll,lunch,02/01/2019,4,04,2019,0,0,0,...,0,0,0,0,0,null,null,null,null,null
1060,95,Chicken Serrano,lunch,02/01/2019,4,04,2019,0,0,0,...,0,0,0,0,0,null,null,null,null,null


In [385]:
# train = {key:val for key, val in trset.items.prevdaysales if val != 42}
train = Trainingset()

for key in trset.arr:
    item = trset.arr[key]
    if item.prevdaysales == [0, 0, 0, 0, 0, 0, 0] or 'null' in item.prevdaysales:
        continue
    if item.weather.weather_m == 'null' or item.weather.weather_a == 'null' \
    or item.weather.temp == 'null' or item.weather.wind == 'null' \
    or item.weather.precip == 'null':
        continue
    else:
        train.set_item(item)

In [387]:
train.items

,id,name,category,date,dayofweek,weeknum,year,holiday,soldnum,prevdaysales1,...,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,prevdaysales7,weather_morning,weather_afternoon,temp,wind,precip
0,67,Mixed Berry Scone,bakedgoods,01/02/2019,2,00,2019,0,17,22,...,14,21,14,28,17,116,119,6,8,0.0
1,68,Pain au Chocolat,bakedgoods,01/02/2019,2,00,2019,0,5,4,...,3,4,4,10,6,116,119,6,8,0.0
2,75,Croissant,bakedgoods,01/02/2019,2,00,2019,0,5,8,...,7,8,6,12,7,116,119,6,8,0.0
3,72,Oatmeal & Raisin Cookie,bakedgoods,01/02/2019,2,00,2019,0,3,2,...,1,10,4,7,3,116,119,6,8,0.0
4,66,Custard Tart,bakedgoods,01/02/2019,2,00,2019,0,4,5,...,6,8,8,10,10,116,119,6,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,66,Custard Tart,bakedgoods,01/31/2019,3,04,2019,0,0,6,...,3,5,7,8,4,122,317,3,21,0.0
703,76,Peanut Butter Brownie,bakedgoods,01/31/2019,3,04,2019,0,0,0,...,2,0,0,3,2,122,317,3,21,0.0
704,79,Ham Cheese Croissant,breakfast,01/31/2019,3,04,2019,0,0,3,...,5,3,2,4,6,122,317,3,21,0.0
705,83,Mediterranean Vegan Wrap,lunch,01/31/2019,3,04,2019,0,0,2,...,1,0,2,2,2,122,317,3,21,0.0


In [390]:
# trset.items.loc[trset.items['date'] == '01/11/2019']